In [56]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import IncrementalPCA
from tabulate import tabulate
import tensorflow as tf
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [57]:

ddi_fp = r"C:\Users\sreej\Desktop\Capstone\drugbank.tab"
ddi = pd.read_csv(ddi_fp, sep='\t')

#kaggle_fp = "Data Files\\SMILES-Kaggle\\chembl_22_clean_1576904_sorted_std_final.smi"
#smiles = pd.read_csv(kaggle_fp, sep='\t')

ddi["Y"] = ddi["Y"].astype("category")
ddi["Map"] = ddi["Map"].astype("category")

#counting interaction types for potential later weighting
interaction_counts = pd.DataFrame(ddi['Y'].value_counts().rename_axis('y').reset_index(name='count')).sort_values(by='count', ascending=False)
interaction_counts['row_num'] = interaction_counts.index + 1
interaction_counts['log_count'] = np.log(interaction_counts['count'])

#listing longer explanations of interaction types for later use
interaction_types = ddi[['Y','Map']].drop_duplicates(subset=['Y'])

#remove longer name of interaction type from main dataset
ddi = ddi.drop("Map",axis=1)

ddi.head(10)

,ID1,ID2,Y,X1,X2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
5,DB11630,DB00460,1,OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
6,DB00553,DB00460,1,COC1=C2OC(=O)C=CC2=CC2=C1OC=C2,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
7,DB06261,DB00460,1,[H]N([H])CC(=O)CCC(=O)OCCCCCC,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
8,DB01878,DB00460,1,O=C(C1=CC=CC=C1)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
9,DB00140,DB00460,1,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...


In [58]:
print(ddi.shape)

(191808, 5)


In [59]:
#quick function to turn a list of size 1 lists of strings into a list of strings, for later use
def delist(list_of_lists):
    list_of_strings = []
    for inner_list in list_of_lists:
        string = inner_list[0]
        list_of_strings.append(string)
    return list_of_strings

Preprocessing

In [60]:

# counting drugs by number of mentions in database
old = pd.DataFrame()
old["total"] = ddi['ID1'].value_counts()
old = old.reset_index()
old.columns = ['ID', 'count'] 
new = pd.DataFrame()
new["total"] = ddi['ID2'].value_counts()
new = new.reset_index()
new.columns = ['ID', 'count'] 
drug_counts = pd.merge(old,new,how='outer',on='ID').fillna(0)
drug_counts['total'] = drug_counts['count_x'] + drug_counts['count_y']

drug_counts = drug_counts.sort_values(by='total')
drug_counts_one = pd.DataFrame(drug_counts[drug_counts['total']==1]['ID'])

#removing drugs only in database once
ddi_proc = ddi[ ~ddi['ID1'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ ~ddi_proc['ID2'].isin(drug_counts_one['ID'])]

#removing one particular drug with a problematic SMILES code
ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


In [61]:
#create main datasets

data = dc.data.NumpyDataset(X=ddi_proc[['X1','X2']], y=ddi[['Y']])
df = data.to_dataframe()
df = df.sample(frac=1).reset_index(drop=True)

X_one = delist(df[["X1"]].values.tolist())
X_two = delist(df[["X2"]].values.tolist())

search_string = "nan"

count = X_one.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_one.")
count = X_two.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_two.")

In [62]:
# reduce dataset down to equal number of top 20 categories to create an equalized dataset

top20 = interaction_counts[interaction_counts['row_num'] <= 20]

reduced_df = df.merge(top20['y'], on='y')
reduced_df=reduced_df.sample(frac=1).reset_index(drop=True)

#eq_df = reduced_df.groupby('y').apply(lambda x: x.sample(min(len(x), 500))).reset_index(drop=True)

#eq_df

In [63]:
#print(df.columns)  
#print(top20.columns)
#print(interaction_counts)
red_df20 = reduced_df[['X1', 'X2', 'y']]
red_df20

,X1,X2,y
0,O[C@@H]1CO[C@@H](O[C@@H]2CO[C@@H](O)[C@H](OS(O...,C[C@@H](CC1=CC(O)=C(O)C=C1)[C@H](C)CC1=CC(O)=C...,6
1,CN1CCN(CCCN2C3=CC=CC=C3SC3=C2C=C(C=C3)C(F)(F)F...,BrC1=CC2=C(NC(=O)CN=C2C2=CC=CC=N2)C=C1,49
2,CCN(CC)CC1=C(O)C=CC(NC2=C3C=CC(Cl)=CC3=NC=C2)=C1,NNCCC1=CC=CC=C1,73
3,CCO,CC(C)NCC(O)COC1=CC=CC=C1OCC=C,16
4,CN1CCN(CC1)C(=O)O[C@@H]1N(C(=O)C2=NC=CN=C12)C1...,CCC1(CCC(=O)NC1=O)C1=CC=CC=C1,49
...,...,...,...
178489,[H][C@@]12CCCN1C(=O)[C@H](CC1=CC=CC=C1)N1C(=O)...,[H][C@@]12CC[C@@](O)(C#C)[C@@]1(C)CC[C@@]1([H]...,49
178490,[H][C@@](CC)(C1=CC(NS(=O)(=O)C2=NC=C(C=C2)C(F)...,COC1=CC=CC=C1OCC(O)CN1CCN(CC(=O)NC2=C(C)C=CC=C...,47
178491,CN1CCN2C(C1)C1=CC=CC=C1CC1=C2N=CC=C1,CC(C)OP(F)(=O)OC(C)C,73
178492,CN1CCN(CCCN2C3=CC=CC=C3SC3=C2C=C(Cl)C=C3)CC1,CC(C)NCC(O)C1=CC=C(NS(C)(=O)=O)C=C1,60


Information

Each atom in a molecule is a node on the graph. 

Lines connecting nodes represent chemical bonds between atoms, with the type of bond potentially encoded as edge attributes. 



In [64]:

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data 
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GINEConv
from torch_geometric.nn import global_mean_pool
import torch.optim as optim
from rdkit import Chem


def smiles_to_graphs1(smiles): 

    # creat RDKit Mol object
    mol = Chem.MolFromSmiles(smiles)

    if mol is None: 
        print(f'Invalid SMILES Code: {smiles}')
        return None

    # atom features
    node_features = []
    for n in mol.GetAtoms(): 
        node_features.append([n.GetAtomicNum()])
    
    # bond features 
    bond_map = {
    Chem.BondType.SINGLE: 1, 
    Chem.BondType.DOUBLE: 2, 
    Chem.BondType.TRIPLE: 3, 
    Chem.BondType.AROMATIC: 4
    } 

    bond_indices = []
    bond_type_features = []
    for b in mol.GetBonds(): 

        # bond types (bi-drectional)
        bond_type = bond_map.get(b.GetBondType(), 0)
        bond_type_features.append([bond_type])
        bond_type_features.append([bond_type])

        # bond indices (bi-drectional)
        a1_idx = b.GetBeginAtomIdx()
        a2_idx = b.GetEndAtomIdx()

        bond_indices.append([a1_idx, a2_idx])
        bond_indices.append([a2_idx, a1_idx])

    # tensors 
    node_tensor = torch.tensor(node_features, dtype = torch.float)
    bond_index_tensor = torch.tensor(bond_indices, dtype = torch.long).t().contiguous()
    bond_type_tensor = torch.tensor(bond_type_features)

    return Data(x = node_tensor, edge_index = bond_index_tensor, edge_attr = bond_type_tensor)




In [65]:
# helper function for process_dataset
def process_row(smiles1, smiles2): 
    mol1, mol2 = Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)
    if not mol1 or not mol2:
        return None
    
    graph1 = smiles_to_graphs1(smiles1)
    graph2 = smiles_to_graphs1(smiles2)

    return graph1, graph2

def process_dataset(red_df20): 
    processed_data = []

    for _, row in red_df20.iterrows(): 
        smiles1, smiles2 = row['X1'], row['X2']
        result = process_row(smiles1, smiles2)
        if result: 
            graph1, graph2 = result 
            processed_data.append((graph1, graph2, row['y']))

    # list of tuples containing two Data objects for the two drug grahps, and the label
    return processed_data



In [78]:
class MolGraphGNN(nn.Module):
    def __init__(self, input_dim = 1, edge_dim = 1, hidden_dim = 128, out_dim = 128):
        super(MolGraphGNN, self).__init__()

        # use GINEConv due to edge_attr handling
        self.conv1 = GINEConv(nn.Linear(input_dim, hidden_dim), edge_dim = edge_dim)
        self.conv2 = GINEConv(nn.Linear(hidden_dim, hidden_dim), edge_dim = edge_dim)
        
    def forward(self, x, edge_index, edge_attr): 
        edge_attr = edge_attr.float()
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)

        x = x.mean(dim = 0)
        return x 
    
class DDIClassifier(nn.Module): 
    def __init__(self, gnn, embedding_dim = 128, num_classes = 20):
        super(DDIClassifier, self).__init__()
        self.gnn = gnn
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, drug1, drug2): 
        emb1 = self.gnn(drug1.x, drug1.edge_index, drug1.edge_attr)
        emb2 = self.gnn(drug2.x, drug2.edge_index, drug2.edge_attr)

        min_batch = min(emb1.shape[0], emb2.shape[0])
        emb1, emb2 = emb1[:min_batch], emb2[:min_batch]

        if emb1.dim() == 1: 
            emb1 = emb1.unsqueeze(0)
        if emb2.dim() == 1: 
            emb2 = emb2.unsqueeze(0)
        
        print(f"emb1 shape: {emb1.shape}, emb2 shape: {emb2.shape}")
        
        x = torch.cat((emb1, emb2), dim = 1)
        print(f"Concatenated x shape: {x.shape}")

        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        print(f"Output x shape: {x.shape}")

        return x



In [67]:
processed_data = process_dataset(red_df20)

[21:06:40] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[21:06:40] SMILES Parse Error: check for mistakes around position 76:
[21:06:40] C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C
[21:06:40] ~~~~~~~~~~~~~~~~~~~~^
[21:06:40] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'
[21:07:42] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[21:07:42] SMILES Parse Error: check for mistakes around position 76:
[21:07:42] C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C
[21:07:42] ~~~~~~~~~~~~~~~~~~~~

In [83]:
processed_data

[(Data(x=[35, 1], edge_index=[2, 72], edge_attr=[72, 1]),
  Data(x=[22, 1], edge_index=[2, 46], edge_attr=[46, 1]),
  6),
 (Data(x=[28, 1], edge_index=[2, 62], edge_attr=[62, 1]),
  Data(x=[19, 1], edge_index=[2, 42], edge_attr=[42, 1]),
  49),
 (Data(x=[25, 1], edge_index=[2, 54], edge_attr=[54, 1]),
  Data(x=[10, 1], edge_index=[2, 20], edge_attr=[20, 1]),
  73),
 (Data(x=[3, 1], edge_index=[2, 4], edge_attr=[4, 1]),
  Data(x=[19, 1], edge_index=[2, 38], edge_attr=[38, 1]),
  16),
 (Data(x=[27, 1], edge_index=[2, 60], edge_attr=[60, 1]),
  Data(x=[16, 1], edge_index=[2, 34], edge_attr=[34, 1]),
  49),
 (Data(x=[30, 1], edge_index=[2, 62], edge_attr=[62, 1]),
  Data(x=[49, 1], edge_index=[2, 110], edge_attr=[110, 1]),
  73),
 (Data(x=[21, 1], edge_index=[2, 48], edge_attr=[48, 1]),
  Data(x=[18, 1], edge_index=[2, 38], edge_attr=[38, 1]),
  47),
 (Data(x=[19, 1], edge_index=[2, 38], edge_attr=[38, 1]),
  Data(x=[25, 1], edge_index=[2, 52], edge_attr=[52, 1]),
  70),
 (Data(x=[20, 1], 

In [82]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(processed_data, test_size = 0.2, random_state = 42, stratify = [label for _, _, label in processed_data])


train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle = False)

print(f"drug1 batch size: {drug1.x.shape[0]}, drug2 batch size: {drug2.x.shape[0]}")
print(f"Labels batch size: {labels.shape[0]}")

drug1 batch size: 727, drug2 batch size: 862
Labels batch size: 32


In [85]:
import torch 
import torch.optim as opti
import torch.nn as nn

gnn_model = MolGraphGNN(input_dim = 1, edge_dim = 1, hidden_dim = 128, out_dim = 128)

ddi_classifier = DDIClassifier(gnn=gnn_model, embedding_dim = 128, num_classes= 20)
optimizer = optim.Adam(ddi_classifier.parameters(), lr = 0.01)

criterion = nn.CrossEntropyLoss()
device = torch.device('cpu')
ddi_classifier.to(device)


DDIClassifier(
  (gnn): MolGraphGNN(
    (conv1): GINEConv(nn=Linear(in_features=1, out_features=128, bias=True))
    (conv2): GINEConv(nn=Linear(in_features=128, out_features=128, bias=True))
  )
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
)

In [86]:

# Training Loop
epochs = 10
for epoch in range(epochs):
    gnn_model.train()  # Set the model to training mode
    running_loss = 0.0
    
    # Iterate through the training batches
    for batch_idx, (drug1, drug2, labels) in enumerate(train_loader):
        # Forward pass
        optimizer.zero_grad()
        outputs = gnn_model(drug1, drug2)
        
        # Compute loss
        loss = criterion(outputs, labels.long())  # Ensure labels are long type
        loss.backward()  # Backpropagation
        
        # Update model weights
        optimizer.step()

        running_loss += loss.item()

    # Log training loss every epoch
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    # Validation after every epoch (optional)
    gnn_model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for drug1, drug2, labels in test_loader:
            outputs = gnn_model(drug1, drug2)
            loss = criterion(outputs, labels.long())
            val_loss += loss.item()

            # Calculate accuracy (if needed)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(test_loader)
        accuracy = 100 * correct / total
        print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")


TypeError: MolGraphGNN.forward() missing 1 required positional argument: 'edge_attr'

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    ddi_classifier.train()
    total_loss = 0 
    correct = 0 
    total = 0

    for batch in train_loader: 
        drug1, drug2, labels = batch
        drug1 = drug1.to(device)
        drug2 = drug2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = ddi_classifier(drug1, drug2)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, pred = torch.max(outputs, dim = 1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = total_loss/len(train_loader) 
    epoch_acc = correct/total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # evaluation phase 
    ddi_classifier.eval()
    test_loss = 0 
    correct = 0 
    total = 0 

    with torch.no_grad(): 
        for batch in test_loader: 
            drug1, drug2, labels = batch
            drug1 = drug1.to(device)
            drug2 = drug2.to(device)
            labels = labels.to(device)


In [81]:
# Initialize the model, optimizer, and criterion


gnn_model = MolGraphGNN(input_dim=1, edge_dim=1, hidden_dim=128, out_dim=128)
ddi_classifier = DDIClassifier(gnn=gnn_model, embedding_dim=128, num_classes=20)  # Adjust num_classes if needed
optimizer = torch.optim.Adam(ddi_classifier.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()  # For binary classification

device = torch.device('cpu')
ddi_classifier.to(device)

num_epochs = 10
for epoch in range(num_epochs): 
    ddi_classifier.train()
    total_loss = 0 
    correct = 0 
    total = 0

    for batch in train_loader: 
        drug1, drug2, labels = batch
        drug1 = drug1.to(device)
        drug2 = drug2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = ddi_classifier(drug1, drug2)

        print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, pred = torch.max(outputs, dim=1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = total_loss / len(train_loader)
    epoch_acc = correct/total
    print(f'Accuracy: {epoch_acc}')

ddi_classifier.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        drug1, drug2, labels = batch
        drug1 = drug1.to(device)
        drug2 = drug2.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = ddi_classifier(drug1, drug2)
        print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")
        
        # Compute loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Compute accuracy
        _, pred = torch.max(outputs, dim=1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)

test_loss /= len(test_loader)
test_acc = correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


emb1 shape: torch.Size([1, 128]), emb2 shape: torch.Size([1, 128])
Concatenated x shape: torch.Size([1, 256])
Output x shape: torch.Size([1, 20])
Outputs shape: torch.Size([1, 20]), Labels shape: torch.Size([32])


ValueError: Expected input batch_size (1) to match target batch_size (32).